In [ ]:
import glob #filenames and pathnames utility
import os   #operating sytem utility

import flowgatenist as flow
#from flowgatenist import gaussian_mixture as nist_gmm
import flowgatenist.batch_process as batch_p

import matplotlib.pyplot as plt
from matplotlib import colors
#from matplotlib.backends.backend_pdf import PdfPages

import numpy as np
import pandas as pd
#from scipy import special
#from scipy import misc

#import pystan
import pickle

import seaborn as sns
sns.set()

%load_ext autoreload
%autoreload 2

%matplotlib inline

Indicate the directory where the data is stored:

In [ ]:
notebook_dir = os.getcwd()
notebook_dir

In [ ]:
main_directory = notebook_dir[:notebook_dir.rfind("\\")]
os.chdir(main_directory)
main_directory

Then look at acatter plots of the buffer blanks to choose which one to use for background subtration:

In [ ]:
glob.glob('*blanks*.fcs')

In [ ]:
os.chdir(main_directory)
back_file = glob.glob('*blanks_A3.fcs')[0]
s_back = flow.io.FCSDataFrame(back_file)
s_back.flow_frame = s_back.flow_frame.loc[s_back.flow_frame['FSC-H'] > 0]
s_back.flow_frame = s_back.flow_frame.loc[s_back.flow_frame['SSC-H'] > 0]
s_back.flow_frame['log_fsc'] = np.log10(s_back.flow_frame['FSC-H'])
s_back.flow_frame['log_ssc'] = np.log10(s_back.flow_frame['SSC-H'])
x_bins = np.linspace(2, 5.5, 200)
y_bins = np.linspace(2.4, 5.5, 200)
x = s_back.flow_frame['log_fsc']
y = s_back.flow_frame['log_ssc']
plt.style.use('classic')
plt.rcParams["figure.figsize"] = [8, 4]
fig, axs = plt.subplots(1, 2)
axs[0].hist2d(x, y, bins=[x_bins, y_bins], rasterized=True)
axs[1].hist2d(x, y, bins=[x_bins, y_bins], norm=colors.LogNorm(), rasterized=True);
s_back.flow_frame.shape

Then run the fcs_to_dataframe batch process:

In [ ]:
batch_p.fcs_to_dataframe(main_directory, blank_file=back_file)

Then the automated background subtraction gating:

In [ ]:
batch_p.background_subtract_gating(data_directory=main_directory,
                                   top_directory='ligafy_flow_analysis_git_repo',
                                   cell_type='DH10B',
                                   num_back_clusters=8,
                                   num_cell_clusters=3,
                                   num_memory_inits=5,
                                   back_init=100,
                                   random_cell_inits=50,
                                   show_plots=True,
                                   trim_alpha=0.90)

Check the plots (also saved in the pdf file output, "cell vs. background gating plots.pdf"), to verify that cell gating was sucessful.
Then run the singlet gating:

In [ ]:
batch_p.singlet_gating(data_directory=main_directory,
                       top_directory='ligafy_flow_analysis_git_repo',
                       cell_type='DH10B',
                       random_singlet_inits=100,
                       show_plots=True)

In [ ]:
coli_files, blank_file_list, bead_file_list = batch_p.auto_find_files()
file_list = coli_files

count_list = []
cell_list = []
singlet_list = []

for file in file_list:
        data = pickle.load(open(file, 'rb'))
        df = data.flow_frame
        n = len(df)
        count_list.append(n)
        n = len(df[df.is_cell])
        cell_list.append(n)
        n = len(df[df.is_singlet])
        singlet_list.append(n)

In [ ]:
plt.rcParams["figure.figsize"] = [6, 4]
fig, axs = plt.subplots()
x = [n+1 for n in range(len(count_list))]
axs.plot(x, count_list, 'o');
axs.plot(x, cell_list, 'o');
axs.plot(x, singlet_list, 'o');
axs.set_yscale('log');